In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('logistic-regression').getOrCreate()

22/12/26 00:34:05 WARN Utils: Your hostname, PSui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.7 instead (on interface en0)
22/12/26 00:34:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/26 00:34:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/26 00:34:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/26 00:34:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/26 00:34:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer # term frequency 약자

# spark 공식문서 예제
## spark detect (스파크가 나오는 경우 1)


In [7]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [8]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [9]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [10]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [11]:
model = pipeline.fit(training)

22/12/26 00:37:47 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/12/26 00:37:47 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/26 00:37:47 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/26 00:37:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/26 00:37:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [12]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])



In [13]:
prediction = model.transform(test)

In [14]:
prediction.select(['id', 'text', 'probability', 'prediction']).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748817...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

